In [1]:
import requests
import os
import pandas as pd
import io
from datetime import datetime
import sqlite3

In [2]:
def retrieve_ssga_etf_holdings(url_endpoint: str, db_name: str='C:/Users/chris/OneDrive/Documents/Jupyter/data/financial_data.db') -> pd.DataFrame:
    url = f'https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/etfs/us/{url_endpoint}' # url_endpoint format is "holdings-daily-us-en-spy.xlsx"
    response = requests.get(url)
    
    # check if the response was successful
    if response.status_code == 200:
        df = pd.read_excel(io.BytesIO(response.content))
        print('Data successfully imported to DataFrame.')
    else:
        print(f'Failed to download the file. Status code=: {response.status_code}')

    ticker = df[df['Fund Name:'] == 'Ticker Symbol:'].iloc[0,1]
    as_of = df[df['Fund Name:'] == 'Holdings:'].iloc[0,1]
    as_of = as_of.split(' ')[-1]
    as_of = datetime.strptime(as_of, '%d-%b-%Y')
    index_name = df.columns[1]
    df = df.drop([0,1,2]).reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df.dropna(inplace=True)
    
    df['as_of'] = as_of
    df['index_ticker'] = ticker
    df['index_name'] = index_name

    column_renames = {
        'Name': 'holding_name',
        'Ticker': 'holding_ticker',
        'Identifier': 'holding_identifier',
        'SEDOL': 'holding_sedol',
        'Weight': 'holding_weight',
        'Sector': 'holding_sector',
        'Shares Held': 'holding_shares_held',
        'Local Currency': 'holding_currency',
        'as_of': 'as_of',
        'index_ticker': 'index_ticker',
        'index_name': 'index_name'
    }

    df.rename(columns=column_renames, inplace=True)

    db_name = 'C:/Users/chris/OneDrive/Documents/Jupyter/data/financial_data.db'
    conn = sqlite3.connect(db_name)
    df.to_sql(name='index_holdings', con=conn, if_exists='append', index=False)
    conn.close()
    return df

### SSGA -- XRT

In [3]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-xrt.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,GAMESTOP CORP CLASS A,GME,36467W109,B0LLFT5,2.173276,-,408155,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
1,ABERCROMBIE + FITCH CO CL A,ANF,002896207,2004185,2.118624,-,50408,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
2,WARBY PARKER INC CLASS A,WRBY,93403J106,BLGZN51,1.915122,-,466295,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
3,BOOT BARN HOLDINGS INC,BOOT,099406100,BRS6600,1.799346,-,66952,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
4,SPROUTS FARMERS MARKET INC,SFM,85208M102,BCGCR79,1.757204,-,94415,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
...,...,...,...,...,...,...,...,...,...,...,...
76,GENESCO INC,GCO,371532102,2367101,0.334752,-,52068,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
77,ARKO CORP,ARKO,041242108,BMH73N1,0.148643,-,109201,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
78,QURATE RETAIL INC SERIES A,QRTEA,74915M100,BZ19HB0,0.115723,-,692104,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF
79,STATE STREET INSTITUTIONAL LIQ,-,964WKM905,-,0.098361,-,417585.14,USD,2024-05-30,XRT,SPDR® S&P® Retail ETF


### SSGA -- SPY

In [4]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-spy.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,MICROSOFT CORP,MSFT,594918104,2588173,6.996602,-,89267969,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
1,APPLE INC,AAPL,037833100,2046251,6.305089,-,174385360,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
2,NVIDIA CORP,NVDA,67066G104,2379504,6.197674,-,29674100,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
3,AMAZON.COM INC,AMZN,023135106,2000019,3.7221,-,109817205,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
4,META PLATFORMS INC CLASS A,META,30303M102,B7TL820,2.33327,-,26430948,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
...,...,...,...,...,...,...,...,...,...,...,...
499,MOHAWK INDUSTRIES INC,MHK,608190104,2598699,0.013835,-,617100,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
500,BIO RAD LABORATORIES A,BIO,090572207,2098508,0.013366,-,250763,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
501,PARAMOUNT GLOBAL CLASS B,PARA,92556H206,BKTNTR9,0.012966,-,5803713,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust
502,FOX CORP CLASS B,FOX,35137L204,BJJMGY5,0.009513,-,1587211,USD,2024-05-30,SPY,SPDR® S&P 500® ETF Trust


### SSGA -- DIA

In [5]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-dia.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,UNITEDHEALTH GROUP INC,UNH,91324P102,2917766,8.292228,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
1,GOLDMAN SACHS GROUP INC,GS,38141G104,2407966,7.751292,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
2,MICROSOFT CORP,MSFT,594918104,2588173,7.139081,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
3,CATERPILLAR INC,CAT,149123101,2180201,5.840628,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
4,HOME DEPOT INC,HD,437076102,2434209,5.66726,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
5,AMGEN INC,AMGN,031162100,2023607,5.182105,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
6,VISA INC CLASS A SHARES,V,92826C839,B2PZN04,4.670781,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
7,MCDONALD S CORP,MCD,580135101,2550707,4.339711,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
8,AMERICAN EXPRESS CO,AXP,025816109,2026082,4.084049,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust
9,SALESFORCE INC,CRM,79466L302,2310525,3.753324,-,5584806,USD,2024-05-30,DIA,SPDR® Dow Jones® Industrial Average℠ ETF Trust


### SSGA -- Tech Select Sector (XLK)

In [6]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-xlk.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,MICROSOFT CORP,MSFT,594918104,2588173,22.513774,-,35492746,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
1,APPLE INC,AAPL,037833100,2046251,21.452163,-,73311607,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
2,NVIDIA CORP,NVDA,67066G104,2379504,5.481217,-,3242714,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
3,BROADCOM INC,AVGO,11135F101,BDZ78H9,4.637651,-,2222553,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
4,ADVANCED MICRO DEVICES,AMD,007903107,2007849,2.829859,-,11094122,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
...,...,...,...,...,...,...,...,...,...,...,...
63,F5 INC,FFIV,315616102,2427599,0.103344,-,403817,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
64,QORVO INC,QRVO,74736K101,BR9YYP4,0.098445,-,662846,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
65,STATE STREET INSTITUTIONAL LIQ,-,964WKM905,-,0.043367,-,28344266.83,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund
66,US DOLLAR,-,999USDZ92,-,0.004113,-,2689077.99,USD,2024-05-30,XLK,The Technology Select Sector SPDR® Fund


### SSGA - Financial Select Sector (XLF)

In [7]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-xlf.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,BERKSHIRE HATHAWAY INC CL B,BRK.B,084670702,2073390,13.158086,-,12291235,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
1,JPMORGAN CHASE + CO,JPM,46625H100,2190385,10.200488,-,19532601,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
2,VISA INC CLASS A SHARES,V,92826C839,B2PZN04,7.596128,-,10686962,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
3,MASTERCARD INC A,MA,57636Q104,B121557,6.457726,-,5575334,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
4,BANK OF AMERICA CORP,BAC,060505104,2295677,4.710691,-,46544860,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
...,...,...,...,...,...,...,...,...,...,...,...
70,INVESCO LTD,IVZ,G491BT108,B28XP76,0.120759,-,3000821,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
71,COMERICA INC,CMA,200340107,2212870,0.115603,-,881433,USD,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
72,EURO CURRENCY,-,999EURZ94,-,0.000004,-,1537.5,EUR,2024-05-30,XLF,The Financial Select Sector SPDR® Fund
73,POUND STERLING,-,999GBPZ94,-,0.000001,-,365,GBP,2024-05-30,XLF,The Financial Select Sector SPDR® Fund


### SSGA - Health Care Select Sector (XLV)

In [8]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-xlv.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,ELI LILLY + CO,LLY,532457108,2516152,12.376769,-,5746889,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
1,UNITEDHEALTH GROUP INC,UNH,91324P102,2917766,8.483333,-,6665766,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
2,JOHNSON + JOHNSON,JNJ,478160104,2475833,6.659844,-,17348956,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
3,MERCK + CO. INC.,MRK,58933Y105,2778844,6.009204,-,18262408,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
4,ABBVIE INC,ABBV,00287Y109,B92SR70,5.255265,-,12723979,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
...,...,...,...,...,...,...,...,...,...,...,...
61,HENRY SCHEIN INC,HSIC,806407102,2416962,0.171634,-,937991,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
62,SOLVENTUM CORP,SOLV,83444M101,BMTQB43,0.153477,-,995447,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
63,DAVITA INC,DVA,23918K108,2898087,0.150515,-,387137,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund
64,BIO RAD LABORATORIES A,BIO,090572207,2098508,0.112041,-,150358,USD,2024-05-30,XLV,The Health Care Select Sector SPDR® Fund


### SSGA - Energy Select Sector (XLE)

In [9]:
retrieve_ssga_etf_holdings('holdings-daily-us-en-xle.xlsx')

Data successfully imported to DataFrame.


,holding_name,holding_ticker,holding_identifier,holding_sedol,holding_weight,holding_sector,holding_shares_held,holding_currency,as_of,index_ticker,index_name
0,EXXON MOBIL CORP,XOM,30231G102,2326618,26.61554,-,88404239,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
1,CHEVRON CORP,CVX,166764100,2838555,17.293675,-,41373350,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
2,CONOCOPHILLIPS,COP,20825C104,2685717,8.430235,-,28102334,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
3,EOG RESOURCES INC,EOG,26875P101,2318024,4.491428,-,13910418,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
4,SCHLUMBERGER LTD,SLB,806857108,2779201,4.099852,-,34048907,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
5,MARATHON PETROLEUM CORP,MPC,56585A102,B3K3L40,3.988012,-,8778237,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
6,PHILLIPS 66,PSX,718546104,B78C4Y8,3.749103,-,10257161,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
7,VALERO ENERGY CORP,VLO,91913Y100,2041364,3.23668,-,8121104,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
8,WILLIAMS COS INC,WMB,969457100,2967181,3.11473,-,29018251,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
9,ONEOK INC,OKE,682680103,2130109,2.933948,-,13896095,USD,2024-05-30,XLE,The Energy Select Sector SPDR® Fund
